In [13]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow

### download and clean data

In [14]:
con_data = pd.read_csv('https://cocl.us/concrete_data')
con_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [15]:
con_data.shape

(1030, 9)

In [16]:
con_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [17]:
con_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### data cleaned

In [18]:
con_data_column=con_data.columns
X = con_data[con_data_column[con_data_column!= 'Strength']]
y = con_data['Strength']
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [19]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### inputs of NN

In [20]:
n_cols= X.shape[1]

# Part D: Increate the number of hidden layers

### A.1: Split the data

In [21]:
from sklearn.model_selection import train_test_split
# split after, in train_and_test function

### A.2: The regression model

In [22]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

def regression_model(network_arch, n_cols, optimizer='adam', loss='mean_squared_error'):
    '''
    network_arch: 
          {"layer_1": {"n_nodes": n1, "activation": activation1},
           ...
           "layer_n": {"n_nodes: nn}
          }
    '''
    model = Sequential()
    n_layers = len(network_arch)
    output_layer = "layer_"+str(n_layers)
    
    for i,layer in network_arch.items():
        if i == "layer_1":
            model.add(Dense(int(layer.get("n_nodes")), activation=layer.get("activation"), input_shape=(n_cols,)))
        elif i == output_layer:
            model.add(Dense(int(layer.get("n_nodes"))))
        else:
            model.add(Dense(int(layer.get("n_nodes")), activation=layer.get("activation")))
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [23]:
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

def train_and_test(X, y, network_arch, n_cols, optimizer , loss, n_times, epochs, norm, seed=360):
    '''
    seed: for spilt data
    norm: (bool) normalization or not
    '''
    mse_list=[]
    for i in range(n_times):
        if norm:
            X= preprocessing.StandardScaler().fit(X).transform(X)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed+i)
        model = regression_model(network_arch=network_arch, n_cols=n_cols)
        model.fit(X_train, y_train, epochs= epochs, verbose=2)
        y_hat= model.predict(X_test)
        mse = mean_squared_error(y_test, y_hat)
        print("time {} : mse= {}".format(i+1,mse))
        mse_list.append(mse)
    return mse_list

### D.3: Part D result


In [25]:
model_archD = {"layer_1": {"n_nodes": 10, "activation": "relu"}, 
               "layer_2": {"n_nodes": 10, "activation": "relu"},
               "layer_3": {"n_nodes": 10, "activation": "relu"},
               "layer_4": {"n_nodes": 1},
                }

In [26]:
mse_listA=train_and_test(X=X, y=y, network_arch=model_archD, n_cols=n_cols, optimizer="adam", loss="mean_squared_error", 
                        n_times=50, epochs=50, norm= True, seed=360)

Epoch 1/50
23/23 - 1s - loss: 1565.0787 - 1s/epoch - 60ms/step
Epoch 2/50
23/23 - 0s - loss: 1536.1228 - 44ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 1505.6589 - 43ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 1464.1198 - 43ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 1399.4817 - 43ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1295.6278 - 43ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1135.6715 - 47ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 912.2820 - 42ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 653.0889 - 43ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 427.8751 - 48ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 301.6518 - 44ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 256.4207 - 43ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 232.3079 - 42ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 214.8890 - 42ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 201.8786 - 44ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 191.1910 - 4

23/23 - 0s - loss: 159.7937 - 43ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 157.8643 - 43ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 155.4890 - 41ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 153.4186 - 44ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 151.8914 - 43ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 150.2270 - 42ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 148.4460 - 42ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 147.0697 - 44ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 145.8598 - 44ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 144.1688 - 43ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 143.2079 - 42ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 141.8223 - 44ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 140.3105 - 45ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 139.3006 - 44ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 138.1607 - 45ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 136.9677 - 42ms/epoch

Epoch 5/50
23/23 - 0s - loss: 1406.8428 - 55ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1307.8252 - 52ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1161.1449 - 49ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 946.9241 - 44ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 687.9700 - 54ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 438.6251 - 53ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 293.9598 - 51ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 240.3059 - 54ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 223.7025 - 55ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 212.2430 - 50ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 202.9973 - 93ms/epoch - 4ms/step
Epoch 16/50
23/23 - 0s - loss: 193.8450 - 56ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 186.4250 - 49ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 179.6692 - 51ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 172.6874 - 55ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 167.3041 - 

Epoch 33/50
23/23 - 0s - loss: 126.3341 - 43ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 124.4947 - 39ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 122.7125 - 40ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 121.0527 - 41ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 119.3677 - 38ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 118.0512 - 38ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 116.7871 - 43ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 115.4695 - 38ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 114.1486 - 38ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 112.9293 - 39ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 111.6018 - 39ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 110.4402 - 45ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 109.7179 - 44ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 108.1231 - 86ms/epoch - 4ms/step
Epoch 47/50
23/23 - 0s - loss: 106.9675 - 55ms/epoch - 2ms/step
Epoch 48/50
23/23 - 0s - loss: 105.8426 

Epoch 10/50
23/23 - 0s - loss: 328.9192 - 74ms/epoch - 3ms/step
Epoch 11/50
23/23 - 0s - loss: 255.6366 - 48ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 237.4761 - 40ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 224.5954 - 42ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 215.4231 - 38ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 207.2277 - 44ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 200.6285 - 45ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 194.5992 - 44ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 189.3176 - 40ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 184.2243 - 42ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 179.6664 - 45ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 175.3961 - 60ms/epoch - 3ms/step
Epoch 22/50
23/23 - 0s - loss: 171.3768 - 44ms/epoch - 2ms/step
Epoch 23/50
23/23 - 0s - loss: 168.3999 - 43ms/epoch - 2ms/step
Epoch 24/50
23/23 - 0s - loss: 164.8034 - 53ms/epoch - 2ms/step
Epoch 25/50
23/23 - 0s - loss: 161.7101 

...

Epoch 10/50
23/23 - 0s - loss: 788.9862 - 41ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 604.0486 - 43ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 452.0408 - 45ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 347.7874 - 41ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 284.8149 - 39ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 251.6407 - 44ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 232.8227 - 41ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 219.2996 - 40ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 210.0630 - 40ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 202.4073 - 40ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 196.0524 - 41ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 190.2054 - 38ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - loss: 185.1457 - 39ms/epoch - 2ms/step
Epoch 23/50
23/23 - 0s - loss: 180.5107 - 38ms/epoch - 2ms/step
Epoch 24/50
23/23 - 0s - loss: 176.3767 - 40ms/epoch - 2ms/step
Epoch 25/50
23/23 - 0s - loss: 172.6986 

Epoch 37/50
23/23 - 0s - loss: 142.6131 - 39ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 140.6982 - 40ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 140.0180 - 41ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 138.5911 - 40ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 137.3666 - 40ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 136.0002 - 40ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 135.2240 - 40ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 133.8921 - 39ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 132.9023 - 43ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 131.7781 - 41ms/epoch - 2ms/step
Epoch 47/50
23/23 - 0s - loss: 130.7520 - 50ms/epoch - 2ms/step
Epoch 48/50
23/23 - 0s - loss: 130.2627 - 54ms/epoch - 2ms/step
Epoch 49/50
23/23 - 0s - loss: 129.0979 - 55ms/epoch - 2ms/step
Epoch 50/50
23/23 - 0s - loss: 128.2975 - 47ms/epoch - 2ms/step
time 41 : mse= 126.24300539745309
Epoch 1/50
23/23 - 1s - loss: 1574.4204 - 961ms/epoch - 42ms/step
Epoc

Epoch 14/50
23/23 - 0s - loss: 202.7045 - 48ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 187.0645 - 48ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 176.2758 - 42ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 168.6217 - 46ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 161.9767 - 43ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 157.7750 - 43ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 153.1881 - 42ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 149.9423 - 44ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - loss: 147.2176 - 45ms/epoch - 2ms/step
Epoch 23/50
23/23 - 0s - loss: 144.7449 - 45ms/epoch - 2ms/step
Epoch 24/50
23/23 - 0s - loss: 142.5002 - 44ms/epoch - 2ms/step
Epoch 25/50
23/23 - 0s - loss: 140.6925 - 48ms/epoch - 2ms/step
Epoch 26/50
23/23 - 0s - loss: 138.9806 - 43ms/epoch - 2ms/step
Epoch 27/50
23/23 - 0s - loss: 137.6645 - 44ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 136.1673 - 43ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 135.0137 

Epoch 41/50
23/23 - 0s - loss: 132.7409 - 45ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 131.9948 - 48ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 130.9655 - 44ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 129.8183 - 46ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 128.8689 - 44ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 128.0854 - 44ms/epoch - 2ms/step
Epoch 47/50
23/23 - 0s - loss: 127.1600 - 42ms/epoch - 2ms/step
Epoch 48/50
23/23 - 0s - loss: 126.2147 - 42ms/epoch - 2ms/step
Epoch 49/50
23/23 - 0s - loss: 125.3265 - 45ms/epoch - 2ms/step
Epoch 50/50
23/23 - 0s - loss: 124.9365 - 41ms/epoch - 2ms/step
time 46 : mse= 138.82185905542215
Epoch 1/50
23/23 - 1s - loss: 1586.2921 - 765ms/epoch - 33ms/step
Epoch 2/50
23/23 - 0s - loss: 1559.2705 - 47ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 1524.2909 - 48ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 1472.8040 - 48ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 1396.5809 - 53ms/epoch - 2ms/step
Epoc

Epoch 18/50
23/23 - 0s - loss: 183.2333 - 43ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 179.1922 - 42ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 175.6086 - 42ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 172.2700 - 45ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - loss: 169.0330 - 43ms/epoch - 2ms/step
Epoch 23/50
23/23 - 0s - loss: 165.8398 - 44ms/epoch - 2ms/step
Epoch 24/50
23/23 - 0s - loss: 162.9691 - 44ms/epoch - 2ms/step
Epoch 25/50
23/23 - 0s - loss: 160.5758 - 44ms/epoch - 2ms/step
Epoch 26/50
23/23 - 0s - loss: 158.4197 - 44ms/epoch - 2ms/step
Epoch 27/50
23/23 - 0s - loss: 156.1314 - 43ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 153.3387 - 45ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 151.4741 - 44ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 149.3751 - 46ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 147.6685 - 48ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 145.6319 - 41ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 143.8040 

In [27]:
print(mse_listA)

[128.73665734571182, 133.2878138894559, 141.5628008817975, 138.0279385379935, 110.490341906372, 116.98522555934272, 89.16706844986216, 129.29064120902473, 128.09626382109002, 109.43488341372668, 152.72394647969466, 130.7182864403664, 121.0474359847104, 121.97343154927773, 153.33866815798888, 120.85820120331496, 95.29621750763468, 111.77041396578905, 146.97861801078295, 125.33490385799054, 129.74761834363898, 130.57560383524248, 125.50841949581454, 138.32300968411838, 110.03187124539171, 142.6116345001268, 107.62780928823234, 109.90503931108363, 142.6713811808842, 122.84904830087764, 136.76612364318316, 144.77832112103954, 134.5143656884011, 99.40526856519153, 133.5974047395325, 128.19774731908308, 130.6641192433034, 132.7705585042575, 117.53941375374276, 124.60620021543826, 126.24300539745309, 143.13278663234638, 136.85539232146246, 135.94007039370015, 131.0199266682128, 138.82185905542215, 111.20838153760121, 127.0376956715437, 143.0492441360772, 127.45780045785688]


In [28]:
df_results = pd.DataFrame.from_dict({"Part": ["D"], "mean mse": [np.mean(mse_listA)], "std mse": [np.std(mse_listA)]})
df_results

,Part,mean mse,std mse
0,D,127.371538,13.946487


## Part D result is better than Part B